In [1]:
import os
import pandas as pd
import numpy as np
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))


In [2]:
tour = validation_data_input.get_data('tour')


In [3]:
# distance to distance bins
tour['distance_bin'] = pd.cut(tour['tautodist'], bins=[0,1,2,5,15,9999],
                                    labels=['0-1', '1-2', '2-5',
                                            '5-15', '15+'])
# Create bins: bins of 2 miles up to 60 miles
max_bin = 60
bin_size = 2
tour['bin_60mi'] = pd.cut(tour['tautodist'], bins=np.arange(0, max_bin+bin_size, bin_size), labels=[str(i) for i in np.arange(0, max_bin, bin_size)])

In [4]:
def plot_bins(df:pd.DataFrame, filter=None, title_name=None):

    if filter is not None:
        df = df.loc[filter]
    df_plot = df.loc[(df['distance_bin'] != float('nan'))].groupby(['source','distance_bin'])['toexpfac'].sum().reset_index()

    df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['toexpfac']. \
        apply(lambda x: 100 * x / float(x.sum()))

    df_plot_ct = df.loc[(df['distance_bin'] != float('nan'))].groupby(['source','distance_bin'])['toexpfac'].count().reset_index().rename(columns={'toexpfac':'sample count'})

    df_plot = df_plot.merge(df_plot_ct, on=['source','distance_bin'])

    if title_name:
        chart_title = "Tour distance by purpose: "+title_name
    else:
        chart_title = "Tour distance"

    fig = px.bar(df_plot, x="distance_bin", y="percentage", color="source",barmode="group",
                facet_col_wrap=2, hover_data=['sample count'],
                title=chart_title)
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.update_layout(height=400, width=700, font=dict(size=11))
    fig.show()

plot_bins(tour)


## Tour Distance by Purpose

In [5]:
plot_bins(tour, tour['pdpurp'] == 1, 'work')

In [6]:
plot_bins(tour, tour['pdpurp'] == 2, 'school')

In [7]:
plot_bins(tour, tour['pdpurp'] == 3, 'escort')

In [8]:
plot_bins(tour, tour['pdpurp'] == 4, 'personal business')

In [9]:
plot_bins(tour, tour['pdpurp'] == 5, 'shopping')

In [10]:
plot_bins(tour, tour['pdpurp'] == 6, 'meal')

In [11]:
plot_bins(tour, tour['pdpurp'] == 7, 'social')